<a href="https://colab.research.google.com/github/jtothehoenderdos/DataScienceMaster/blob/master/BigData_Swarm_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Batch Implementation**

Installing the neccsary libraries and import those

In [0]:
!apt install proj-bin libproj-dev libgeos-dev
!pip install -U git+https://github.com/matplotlib/basemap.git
!pip install ipynb

In [0]:
from pprint import pprint
import os
from ftplib import FTP
!pip install cdflib
import cdflib                   # More info and documentation on CDF Library here: https://github.com/MAVENSDC/cdflib
import zipfile

In [0]:
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt

Below the functions we use for data collection<br>
- We first **get** the required ZIP files
- Afterwards we **extract** the ZIP files
- Then we **select** the data needed to run the application

In [0]:
def donwloadDataZIP():
    ftp = FTP("swarm-diss.eo.esa.int")
    # ftp://swarm-diss.eo.esa.int/Level2daily/Entire_mission_data/TEC/TMS/Sat_A/SW_OPER_TECATMS_2F_20131125T000000_20131125T235959_0301.ZIP
    ftp.login()
    ftp.retrlines("LIST")

    ftp.cwd("Level2daily/Entire_mission_data/TEC/TMS/Sat_A/")
    ftp.retrlines("LIST")

    listing = []
    ftp.retrlines("LIST", listing.append)
    words = listing[0].split(None, 8)
    filename = words[-1].lstrip()
#     print(filename)
    #
    # download the file
    if not os.path.exists("downloadedZIPs"):
        os.makedirs("downloadedZIPs")
    local_filename = os.path.join(r"downloadedZIPs", 'firstDownload.zip')
    lf = open(local_filename, "wb")
    ftp.retrbinary("RETR " + filename, lf.write, 8 * 1024)
    lf.close()


def extractZIP():
    zip_ref = zipfile.ZipFile('downloadedZIPs/firstDownload.zip', 'r')
    zip_ref.extractall('extractedDownloads/')
    zip_ref.close()

def cdfStuff():
    cdf_file = cdflib.CDF('extractedDownloads/SW_OPER_TECATMS_2F_20131125T000000_20131125T235959_0301.cdf')
    cdf_info = cdf_file.cdf_info();
#     pprint(cdf_info)

    variable_lookup = 'Longitude'
    attribute_lookup = 'DESCRIPTION'

#     print('\n ', variable_lookup, 'attget :')
    var_att_get = cdf_file.varattsget(variable=variable_lookup, expand=False);
#     pprint(var_att_get)

#     print('\n ', variable_lookup,':')
    variable_inq = cdf_file.varinq(variable_lookup);
#     pprint(variable_inq)

#     print('\n ', variable_lookup, ' var value:')

    lon_list = cdf_file.varget(variable=variable_lookup);
#     pprint(lon_list)
#     print(len(lon_list))


    lat_list = cdf_file.varget(variable='Latitude');
#     pprint(lat_list)
#     print(len(lat_list))

    Radius_list = cdf_file.varget(variable='Radius');

  

    L1_list = cdf_file.varget(variable='L1');
#    pprint(L1_list)
#     print(len(L1_list))
#     print(min(L1_list))
#     print(max(L1_list))


    time_stamp_list = cdf_file.varget(variable='Timestamp');
#     pprint(time_stamp_list)
#     print(len(time_stamp_list))

    return(lon_list, lat_list, L1_list, time_stamp_list, Radius_list);

In [0]:
print('Download Data ZIP file')
donwloadDataZIP()
print('Extract the downloaded ZIP file')
extractZIP()
print('Do CDF stuff')
cdfStuff()

In [0]:
# setup Lambert Conformal basemap.
m = Basemap(projection='mill', lon_0=0)

# draw coastlines.
m.drawcoastlines()

# draw a boundary around the map, fill the background.
# this background will end up being the ocean color, since
# the continents will be drawn on top.
m.drawmapboundary(fill_color='aqua', zorder=0)

# fill continents, set lake color same as ocean color.
m.fillcontinents(color='coral',lake_color='aqua', zorder=1)

lon_list, lat_list, L1_list, time_stamp_list, radius_list = cdfStuff();

# print(time_stamp_list[1])
xpt,ypt = m(lon_list, lat_list);
zpt = L1_list;
# print(type(xpt))
xs = []
xy = []
# m.pcolor(xpt, ypt, zpt)

m.scatter(xpt, ypt, c=zpt,  cmap='plasma', marker='o', zorder=2, alpha=0.1)
plt.show()

In [0]:

plt.hist(zpt, bins='auto')
plt.show()

xpt
ypt

In [0]:
# setup Lambert Conformal basemap.
m2 = Basemap(projection='mill', lon_0=0)

# draw coastlines.
m2.drawcoastlines()

# draw a boundary around the map, fill the background.
# this background will end up being the ocean color, since
# the continents will be drawn on top.
m2.drawmapboundary(fill_color='aqua', zorder=0)

# fill continents, set lake color same as ocean color.
m2.fillcontinents(color='coral',lake_color='aqua', zorder=1)

lon_list, lat_list, L1_list, time_stamp_list, radius = cdfStuff();

xpt,ypt = m(lon_list, lat_list)
zpt = L1_list

# print(type(xpt))
xs = []
xy = []
# m.pcolor(xpt, ypt, zpt)

# only show abnormal values (with a deviation of the std)
xpt = np.compress(zpt - np.mean(zpt) < 2 * np.std(zpt), xpt)
ypt = np.compress(zpt - np.mean(zpt) < 2 * np.std(zpt), ypt)
zpt = np.compress(zpt - np.mean(zpt) < 2 * np.std(zpt), zpt)

m2.scatter(xpt, ypt, c=zpt,  cmap='plasma', marker='o', zorder=2, alpha=0.5)
plt.show()

In [0]:
lon_list, lat_list, L1_list, time_stamp_list, Radius_list = cdfStuff();
import pandas as pd
sataliteDF = pd.DataFrame(
    {'Latitude': lat_list,
     'Longitude': lon_list,
     'timeStampList': time_stamp_list,
     'Radius': Radius_list
    })

uniekewaardenDF = sataliteDF.drop_duplicates()

len(uniekewaardenDF)

uniekewaardenDF.iloc[1]

In [0]:
from math import sin, cos, sqrt, atan2, radians
i = 14
j = 7
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
    R = 6371 # Radius of the earth in km + the height of the stalite
    print(type(lat1), type(lon1), type(lat2), type(lon2))
    dLat = np.radians(lat2-lat1)
    dLon = np.radians(lon2-lon1)
    rLat1 = np.radians(lat1)
    rLat2 = np.radians(lat2)
    a = np.sin(dLat/2) * np.sin(dLat/2) + np.cos(rLat1) * np.cos(rLat2) * np.sin(dLon/2) * np.sin(dLon/2) 
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c # Distance in km
    return d

def calc_velocity(dist_km, time_start, time_end):
    """Return 0 if time_start == time_end, avoid dividing by 0"""
    print(time_start, time_end)
    return dist_km / ((time_end - time_start)/1000) if time_end > time_start else 0

dist_in_km = getDistanceFromLatLonInKm(lat_list[j],lon_list[j],lat_list[i],lon_list[i])


print(calc_velocity(dist_in_km, uniekewaardenDF['timeStampList'].iloc[0], uniekewaardenDF['timeStampList'].iloc[1]))

In [0]:
%%capture
uniekewaardenDF['ID'] = 1


# First sort by ID and timestamp:
df = uniekewaardenDF.sort_values(by=['ID', 'timeStampList'])

# Group the sorted dataframe by ID, and grab the initial value for lat, lon, and time.
df['lat0'] = df.groupby('ID')['Latitude'].transform(lambda x: x.iat[0])
df['lon0'] = df.groupby('ID')['Longitude'].transform(lambda x: x.iat[0])
df['t_prev'] = df['timeStampList'].shift()

# create a new column for distance
df['dist_km'] = df.apply(
    lambda row: getDistanceFromLatLonInKm(
        lat1=row['Latitude'],
        lon1=row['Longitude'],
        lat2=row['lat0'],
        lon2=row['lon0']
    ),
    axis=1
)

df["dist_km_diff"] = abs(df["dist_km"].diff(-1))

# create a new column for velocity
df['velocity_km_ps'] = df.apply(
    lambda row: calc_velocity(
        dist_km=row['dist_km_diff'],
        time_start=row['t_prev'],
        time_end=row['timeStampList']
    ),
    axis=1
)



In [0]:
from math import sqrt
G = float(6.67408E-11)
M = float(5.972E24)
r = (6371000 + 450000)
vs_ms = sqrt((G*M)/r)
vs_km = vs_ms/1000
print(vs_km)
df['minimunSpeed']= vs_km

In [0]:
df.describe()

In [0]:
Q1= df['velocity_km_ps'].quantile(0.25)
Q3= df['velocity_km_ps'].quantile(0.75)
IQR = Q3 - Q1

lowertreshold =  Q1 - 1.5*IQR
higertreshold = Q3+ 1.5*IQR



dfSTD = df.drop(df[(df.velocity_km_ps > lowertreshold) & (df.velocity_km_ps < higertreshold) | (df.velocity_km_ps > df.minimunSpeed) ].index)



In [0]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 1)
axes.set_title("Hammer projection")
# setup Lambert Conformal basemap.
m3 = Basemap(projection='mill', lon_0=0, ax=axes)

# draw coastlines.
m3.drawcoastlines()

# draw a boundary around the map, fill the background.
# this background will end up being the ocean color, since
# the continents will be drawn on top.
m3.drawmapboundary(fill_color='aqua', zorder=0)

# fill continents, set lake color same as ocean color.
m3.fillcontinents(color='coral',lake_color='aqua', zorder=1)


xpt,ypt = m(dfSTD['Longitude'].values, dfSTD['Latitude'].values)
speed = dfSTD['velocity_km_ps'].values

# print(type(xpt))
xs = []
xy = []
# m.pcolor(xpt, ypt, zpt)

# only show abnormal values (with a deviation of the std)
#xpt = np.compress(zpt - np.mean(zpt) < 2 * np.std(zpt), xpt)
#ypt = np.compress(zpt - np.mean(zpt) < 2 * np.std(zpt), ypt)
#zpt = np.compress(zpt - np.mean(zpt) < 2 * np.std(zpt), zpt)
m3.scatter(xpt, ypt, c=speed,  cmap='plasma', marker='o', zorder=2, alpha=0.5)

plt.savefig('test.png')


In [0]:
 dfSTD.to_csv('rawdata.csv')

In [0]:
if dfSTD.empty:
  # send_attachment.py
  # import necessary packages
  from email.mime.multipart import MIMEMultipart
  from email.mime.text import MIMEText
  from email.mime.image import MIMEImage
  import smtplib
 
  # create message object instance
  msg = MIMEMultipart()


  # setup the parameters of the message
  password = NEEDSTOBEFILLEDIN
  msg['From'] = "satalitedefectdata@gmail.com"
  msg['To'] = "jophoenderdos@hotmail.com"
  msg['Subject'] = "Satalite is working!"
  email_content = """

  Dear Indika,

  We have some great news.
  
  We analyse data about the satalite, and we could not find any problems with the satalite!
  
  
  You can reach us by phone or by mail.

  Mail: satalitedefectdata@gmail.com
  Phone: 00316123456789

  Thank you,

  Group 20. 


  """

  msg.attach(MIMEText(email_content, 'plain'))

  # create server
  server = smtplib.SMTP('smtp.gmail.com: 587')

  server.starttls()

  # Login Credentials for sending the mail
  server.login(msg['From'], password)


  # send the message via the server.
  server.sendmail(msg['From'], msg['To'], msg.as_string())

  server.quit()

  #print "successfully sent email to %s:" % (msg['To'])
  
  
  
else:
   # send_attachment.py
  # import necessary packages
  from email.mime.multipart import MIMEMultipart
  #from email.MIMEImage import MIMEImage
  from email.mime.text import MIMEText
  from email.mime.image import MIMEImage
  from email.mime.base import MIMEBase
  import smtplib
  import mimetypes
 
  # create message object instance
  msg = MIMEMultipart()


  # setup the parameters of the message
  password = NEEDSTOBEFILLEDIN
  msg['From'] = "satalitedefectdata@gmail.com"
  msg['To'] = "jophoenderdos@hotmail.com"
  msg['Subject'] = "IMPORTANT! Satelite Defect!"
  fileToSend = "rawdata.csv"
  email_content = """
  

  Dear,

  We found some major issue with the satalite velocity. 
  This may indicate that the satalite is defect

  Please find enclosed a plot with the locations at which annomalous velocity occured.

  Additonly, a csv file is included with these points. 

  You can reach us by phone or by mail.

  Mail: satalitedefectdata@gmail.com
  Phone: 00316123456789

  Thank you,

  Jop Hoenderdos


  """
  

  
  ctype, encoding = mimetypes.guess_type(fileToSend)
  if ctype is None or encoding is not None:
    ctype = "application/octet-stream"

  maintype, subtype = ctype.split("/", 1)

  if maintype == "text":
    fp = open(fileToSend)
    # Note: we should handle calculating the charset
    attachment = MIMEText(fp.read(), _subtype=subtype)
    fp.close()
  elif maintype == "image":
    fp = open(fileToSend, "rb")
    attachment = MIMEImage(fp.read(), _subtype=subtype)
    fp.close()
  elif maintype == "audio":
    fp = open(fileToSend, "rb")
    attachment = MIMEAudio(fp.read(), _subtype=subtype)
    fp.close()
  else:
    fp = open(fileToSend, "rb")
    attachment = MIMEBase(maintype, subtype)
    attachment.set_payload(fp.read())
    fp.close()
    encoders.encode_base64(attachment)
  attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
  msg.attach(attachment)
  msg.attach(MIMEText(email_content, 'plain'))

  img_data = open('test.png', 'rb').read()

  #attach image to message body
  msg.attach(MIMEImage(img_data))


  # create server
  server = smtplib.SMTP('smtp.gmail.com: 587')

  server.starttls()

  # Login Credentials for sending the mail
  server.login(msg['From'], password)


  # send the message via the server.
  server.sendmail(msg['From'], msg['To'], msg.as_string())

  server.quit()

  print ("successfully sent email")